<h1>Tagging unmarked Ohio trails using data</h1>
<h4>12/4/24</h4>
<h4>Jacob Nicholson</h4>
<p>Many applications hikers use to tour American national parks rely on data from Open Street Maps, an open source editable map. These applications often translate Data from OSM(OpenSteetMaps) in a somewhat unfiltered way that displays private and unoffical trails as equal to recomened and offical trails maintained by the National Park. This reportedly causes users of trail apps derived from OSM such as All Trails to traverse these trails, putting themseleves and the wildilife in harms way. Becuase of this, OSM created the Trial Stewardship Initative in 2021, as a mission for volunteers to correctly tag the trails in national parks accordingly and encourage the applications to display the trails differntly. At the moment, this tagging is done by hand and tagging an entire park for a team takes months. This report is hoping to find ways to automate this tagging, starting with Ohio using existing data from OSM's overpass API and ODNR's(Ohio Department of National Recources) ArcGIS REST Service. </p>

<p>For more information on thh Trail Stewardship Initiative this presentation by OSM is availilbe: <a>https://openstreetmap.us/events/state-of-the-map-us/2024/osm-us-trails-stewardship-initiative/</a></p>

In [2]:
import urllib
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import base64
from IPython.display import GeoJSON
with open('export.geojson') as fo:
    data_str = fo.read()
    data = json.loads(data_str) 
df = pd.DataFrame(data)

I entered this search in the turbo wizard on https://overpass-turbo.eu/, Overpass being Open Street Maps API in order to revieve all listed paths in Ohio

/*
This has been generated by the overpass-turbo wizard.
The original search was:
“highway=path in "Ohio"”
*/
[out:json][timeout:25];
// fetch area “Ohio” to search in
{{geocodeArea:Ohio}}->.searchArea;
// gather results
nwr["highway"="path"](area.searchArea);
// print results
out geom;

I then rendered this file using jupyterlab-geojson 3.4.0

In [3]:
geojson_file = 'export.geojson'
# Load the GeoJSON file
with open(geojson_file, 'r') as f:
    geojson_data = json.load(f)

# Render the GeoJSON
#GeoJSON(data=geojson_data)

Entered an SQL Query to https://gis2.ohiodnr.gov/arcgis/rest/services/OIT_Services/Trails_WayPoints_POI/MapServer/2

Where: 1=1

Out Fields: *

Return Geometry: true

Format: GEOjson

resultOffset: 0

resultRecordCount: 1000

In [4]:
ONDRgeojson_file = 'query.geojson'
# Load the GeoJSON file
with open(ONDRgeojson_file, 'r') as f:
    geojson_data = json.load(f)

# Render the GeoJSON
#GeoJSON(data=geojson_data)

Using this Geokit tool: https://developmentseed.org/geokit-doc-seed/usage/difference-between-two-geojson-files/
and Docker: https://www.docker.com/

I was able to run this code in my terminal: 

docker run --rm -v /Users/jacobnicholson/DataInEmergingMediaAndTech:/mnt/data developmentseed/geokit:python.latest geo difference \
  --geojson_input=export.geojson \
  --geojson_dif=query.geojson \
  --key="trail name" \
  --geojson_output=difference_output.geojson

And output a file that removes all overlap between the two files

In [5]:
Differencegeojson_file = 'difference_output.geojson'
# Load the GeoJSON file
with open(Differencegeojson_file, 'r') as f:
    geojson_data = json.load(f)

# Render the GeoJSON
#GeoJSON(data=geojson_data)

I then organized and color coded this data set of trails based on the Trails Stewardship Initiative recomended nomenclature. 

In [6]:
import folium
from folium.plugins import FloatImage

# Path to your GeoJSON file
Differencegeojson_file = "difference_output.geojson"

# Load the GeoJSON data
with open(Differencegeojson_file, 'r') as file:
    geojson_data = json.load(file)

# Define a function to style the GeoJSON features dynamically
def style_function(feature):
    properties = feature.get('properties', {})
    if properties.get('informal') == 'no' and properties.get('name'):
        # Official Trail
        return {'color': '#1f78b4'}  # Blue
    elif properties.get('informal') == 'yes':
        # Social Trail
        return {'color': '#33a02c'}  # Green
    elif properties.get('access') == 'no':
        # Closed Trail
        return {'color': '#e31a1c'}  # Red
    else:
        # Default style for other trails
        return {'color': '#6a3d9a'}  # Purple

# Initialize a map centered on Ohio
m = folium.Map(location=[40.3675, -82.9962], zoom_start=8, tiles="cartodb positron")

# Add the GeoJSON data to the map with the style function
folium.GeoJson(
    geojson_data,
    style_function=style_function
).add_to(m)

# Add a legend (HTML and CSS)
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 200px;
    height: 150px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
">
    <b>Trail Legend</b><br>
    <i style="background: #1f78b4; width: 18px; height: 18px; display: inline-block;"></i> Official Trail<br>
    <i style="background: #33a02c; width: 18px; height: 18px; display: inline-block;"></i> Social Trail<br>
    <i style="background: #e31a1c; width: 18px; height: 18px; display: inline-block;"></i> Closed Trail<br>
    <i style="background: #6a3d9a; width: 18px; height: 18px; display: inline-block;"></i> Other Trail
</div>
"""
#m.get_root().html.add_child(folium.Element(legend_html))

# Save to an HTML file or display in a Jupyter Notebook
#m
